In [1]:
%reset -f
%matplotlib inline
import numpy as np
import lib.io.stan
import lib.plots.stan
import matplotlib.pyplot as plt
import os 

In [2]:
data_dir = 'datasets/id001_ac'
results_dir = 'results/exp10/exp10.9'
os.makedirs(results_dir,exist_ok=True)
os.makedirs(f'{results_dir}/logs',exist_ok=True)
os.makedirs(f'{results_dir}/figures',exist_ok=True)

network = np.load(f'{data_dir}/AC_network.npz')
SC = network['SC']
K = np.max(SC)
SC = SC / K
SC[np.diag_indices(SC.shape[0])] = 0
gain_mat = network['gain_mat']

slp = np.load(f'{data_dir}/AC_fit_trgt.npz')['fit_trgt']
slp_ds = slp[0:-1:20,:]
snsr_pwr = np.sum(slp_ds**2, axis=0)

In [3]:
nn = SC.shape[0]
ns = gain_mat.shape[0]
nt = slp_ds.shape[0]
I1 = 3.1
tau0=30
time_step=0.1

stan_fname = 'vep-snsrfit-ode'
# lib.io.stan.create_process(['bash','/home/anirudhnihalani/scripts/stancompile.sh', stan_fname],block=True)

# x0_star = np.zeros(nn)
# x_init_star = np.zeros(nn)
# z_init_star = np.zeros(nn)
# amplitude_star = 0.0
# offset = 0.0
# time_step_star = 0.0
# K_star = 0.0
# tau0_star = 0.0

# param_init = {'x0_star':x0_star, 'x_init_star':x_init_star, 'z_init_star':z_init_star,
#               'amplitude_star':amplitude_star, 'offset':offset, 'time_step_star':time_step_star,
#               'K_star':K_star, 'tau0_star':tau0_star}
# param_init_file = 'param_init.R'
# lib.io.stan.rdump(f'{results_dir}/Rfiles/param_init.R',param_init)

epsilon_snsr_pwr = 1.0
for epsilon_slp in np.array([0.1]):
    fname_suffix = f'epsslp{epsilon_slp:0.5f}_epssnsrpwr_{epsilon_snsr_pwr:0.5f}'
    data = {'nn':nn, 'ns':ns, 'nt':nt, 'I1':I1, 'tau0':tau0, 'time_step':time_step,
            'SC':SC, 'gain': gain_mat, 'epsilon_slp':epsilon_slp,
            'epsilon_snsr_pwr':epsilon_snsr_pwr, 'slp':slp_ds, 'snsr_pwr':snsr_pwr}
    input_Rfile = f'fit_data_snsrfit_ode_{fname_suffix}.R'
    os.makedirs(f'{results_dir}/Rfiles',exist_ok=True)
    lib.io.stan.rdump(f'{results_dir}/Rfiles/{input_Rfile}',data)
#     nchains = 4
#     with open('vep-snsrfit-ode.sh','r') as fd:
#         slurm_script = fd.read().format(f'{results_dir}/Rfiles', results_dir, input_Rfile, nchains, eps)
#     with open(f'tmp/vep-snsrfit-ode-eps{eps:0.1f}.sh','w') as fd:
#         fd.write(slurm_script)
#     lib.io.stan.create_process(['sbatch',f'tmp/vep-snsrfit-ode-eps{eps:0.1f}.sh'],block=False)

# data = {'nn':nn, 'ns':ns, 'nt':nt, 'I1':I1, 'time_step':time_step, 'nsteps':nsteps,
#         'tau0':2857.0, 'SC':SC, 'K':K, 'gain': gain_mat, 'epsilon':epsilon, 
#         'slp':syn_data['fit_trgt']}
# input_Rfile = f'fit_data_snsrfit_ode_epsilon{epsilon:0.5f}.R'
# os.makedirs(f'{results_dir}/Rfiles',exist_ok=True)
# lib.io.stan.rdump(f'{results_dir}/Rfiles/{input_Rfile}',data)

# ! srun -p dev --ntasks=1 {stan_fname} sample save_warmup=1 num_warmup=200 num_samples=200 \
# adapt delta=0.8 algorithm=hmc engine=nuts max_depth=10 data file={results_dir}/Rfiles/{input_Rfile} \
# init=0 output file={results_dir}/samples_epsinfer_chain1.csv refresh=5


# cmd = f'./{stan_fname} variational algorithm=meanfield iter=100000 tol_rel_obj=0.01 \
# output_samples=1000 data file={results_dir}/Rfiles/{input_Rfile} \
# init={results_dir}/Rfiles/param_init.R \
# output file={results_dir}/samples_epsilon{epsilon:0.5f}.csv'
# print(cmd.split(' '))
# lib.io.stan.create_process(cmd.split(' '), block=True)



In [6]:
%%bash -s "$stan_fname"
stancompile.sh $1

/home/anirudh/Academia/projects/vep.stan

--- Translating Stan model to C++ code ---
bin/stanc  /home/anirudh/Academia/projects/vep.stan/vep-snsrfit-ode.stan --o=/home/anirudh/Academia/projects/vep.stan/vep-snsrfit-ode.hpp
Model name=vep_snsrfit_ode_model
Input file=/home/anirudh/Academia/projects/vep.stan/vep-snsrfit-ode.stan
Output file=/home/anirudh/Academia/projects/vep.stan/vep-snsrfit-ode.hpp

--- Linking C++ model ---
g++ -Wall -I . -isystem stan/lib/stan_math/lib/eigen_3.3.3 -isystem stan/lib/stan_math/lib/boost_1.64.0 -isystem stan/lib/stan_math/lib/cvodes_2.9.0/include -std=c++1y -DBOOST_RESULT_OF_USE_TR1 -DBOOST_NO_DECLTYPE -DBOOST_DISABLE_ASSERTS -DBOOST_PHOENIX_NO_VARIADIC_EXPRESSION -Wno-unused-function -Wno-uninitialized -I src -isystem stan/src -isystem stan/lib/stan_math/ -DFUSION_MAX_VECTOR_SIZE=12 -Wno-unused-local-typedefs -DEIGEN_NO_DEBUG -DNO_FPRINTF_OUTPUT -pipe    -O3 -o /home/anirudh/Academia/projects/vep.stan/vep-snsrfit-ode src/cmdstan/main.cpp -include /home

In [7]:
%%bash -s "$stan_fname" "$results_dir" "$input_Rfile" "$fname_suffix"

STAN_FNAME=$1
RESULTS_DIR=$2
INPUT_RFILE=$3
# SIGMA=$(printf "%.5f" ${3})
# EPSILON_SLP=$(printf "%.5f" ${4})
# EPSILON_SNSR_PWR=$(printf "%.5f" ${5})
FNAME_SUFFIX=$4
for i in {1..4};
do
./${STAN_FNAME}  id=$((100*${i})) variational iter=1000000 tol_rel_obj=0.01 \
output_samples=1000 data file=${RESULTS_DIR}/Rfiles/${INPUT_RFILE} \
output file=${RESULTS_DIR}/samples_${FNAME_SUFFIX}_chain${i}.csv \
&> ${RESULTS_DIR}/logs/snsrfit_cntr_${FNAME_SUFFIX}_chain${i}.log &
done

In [ ]:
import lib.io.stan
# import importlib
# importlib.reload(lib.io.stan)

csv_fname = 'results/exp10/exp10.4/samples_epsinfer_chain1.csv'
nwarmup = 200
nsampling = 100
ignore_warmup = True
variables_of_interest = ['lp__','accept_stat__','stepsize__','treedepth__','n_leapfrog__',\
                         'divergent__', 'energy__','x0',  'x', 'mu_seeg', 'amplitude', 'offset',\
                         'x_init', 'z_init', 'epsilon']
pstr_samples_1 = lib.io.stan.read_samples(csv_fname,nwarmup,nsampling,ignore_warmup,\
                                          variables_of_interest) # read sampler diagnostics and x0 for all sampling iterations

# csv_fname = 'results/exp10/exp10.4/samples_eps0.1_chain1.csv'
# nwarmup = 1000
# nsampling = 1000
# ignore_warmup = True
# variables_of_interest = ['x','z']
# pstr_samples_2 = lib.io.stan.read_samples(csv_fname,nwarmup,nsampling,ignore_warmup,variables_of_interest) # read 10 samples of hidden state variables x and z

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.violinplot(pstr_samples_1['x0'][:,:]);
xtick_labels = []
for i in range(84):
    if(i%2 == 0):
        xtick_labels.append(str(i+1))
    else:
        xtick_labels.append('')
plt.xticks(np.r_[1:85],xtick_labels);
plt.xlabel('Region#',fontsize=15);
plt.ylabel('$x_0$',fontsize=15);

# Plot the HMC convergence diagnostics
plt.figure(figsize=(20,10))
plt.subplot(4,2,1)
plt.plot(pstr_samples_1['lp__'][40:])
plt.xlabel('Iteration')
plt.ylabel('log prob.')

plt.subplot(4,2,2)
plt.plot(pstr_samples_1['energy__'])
plt.xlabel('Iteration')
plt.ylabel('energy')

plt.subplot(4,2,3)
plt.plot(pstr_samples_1['accept_stat__'])
plt.xlabel('Iteration')
plt.ylabel('accept stat.')

plt.subplot(4,2,4)
plt.plot(pstr_samples_1['stepsize__'])
plt.xlabel('Iteration')
plt.ylabel('step size')

plt.subplot(4,2,5)
plt.plot(pstr_samples_1['treedepth__'])
plt.xlabel('Iteration')
plt.ylabel('tree depth')

plt.subplot(4,2,6)
plt.plot(pstr_samples_1['n_leapfrog__'])
plt.xlabel('Iteration')
plt.ylabel('n_leapfrog')

plt.subplot(4,2,7)
plt.plot(pstr_samples_1['divergent__'])
plt.xlabel('Iteration')
plt.ylabel('divergent')

plt.tight_layout();  

# Mean and 2*std of source activity(x) estimated from posterior samples
plt.figure(figsize=(15,20))
x_mean = np.mean(pstr_samples_1['x'], axis = 0)
x_std = np.std(pstr_samples_1['x'], axis = 0)
nt = x_mean.shape[0]
nn = x_mean.shape[1]
for i in range(nn):
    plt.plot(x_mean[:,i]+4*i)
    plt.fill_between(np.r_[0:nt], x_mean[:,i] - 2*x_std[:,i] + 4*i, x_mean[:,i] + 2*x_std[:,i] + 4*i,alpha=0.1)
plt.title('source activity(x)',fontsize=15);
plt.xlabel('time',fontsize=15);
plt.ylabel('Region#',fontsize=15);
plt.yticks(np.mean(x_mean,axis=0) + 4*np.r_[0:nn], np.r_[1:nn+1]);

In [ ]:
# ntwrk = np.load('datasets/id001_ac/AC_network.npz')
# gain = ntwrk['gain_mat']
# src_sig = pstr_samples_1['x']
# amplitude = pstr_samples_1['amplitude']
# offset = pstr_samples_1['offset']
# seeg = np.zeros([src_sig.shape[0],src_sig.shape[1],gain.shape[0]])
# for i,sample in enumerate(src_sig):
# #     seeg[i] = amplitude[i,:] * ((gain @ sample.T).T + offset[i,:]);
#     seeg[i] = amplitude[i,:] * ((gain @ sample.T).T + offset[i,:])

mdld_data = np.load('datasets/id001_ac/AC_fit_trgt.npz')
plt.figure(figsize=(15,4))
plt.plot(mdld_data['fit_trgt'],'k',alpha=0.4);
plt.plot((pstr_samples_1['mu_seeg'][-10,:,:]),'r', alpha=0.4);

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(pstr_samples_1['x'][-100,:,:]);

In [ ]:
plt.figure(figsize=(13,4))
plt.subplot(131)
plt.hist(pstr_samples_1['amplitude'].flatten())
plt.title('Amplitude')
plt.subplot(132)
plt.hist(pstr_samples_1['offset'].flatten())
plt.title('Offset')
plt.subplot(133)
plt.plot(pstr_samples_1['epsilon'])
plt.title('epsilon')
